In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()

In [6]:
data

Data(adj_t=[169343, 169343, nnz=2315598], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [18]:
n_iters = 3000
epochs = 10000
log_steps = 100
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [19]:
class OneLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim):
        super(OneLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        
        self.convs.append(SAGEConv(input_dim, output_dim))
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        return x.log_softmax(dim=-1)

In [20]:
model = OneLayer(input_dim, hidden_channels, output_dim).to(device)

In [21]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [22]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 2.6279, Train: 26.40%, Valid: 25.30% Test: 22.93%
Epoch: 200, Loss: 2.2587, Train: 40.51%, Valid: 42.20% Test: 40.24%
Epoch: 300, Loss: 2.0079, Train: 48.94%, Valid: 50.53% Test: 49.19%
Epoch: 400, Loss: 1.8331, Train: 53.56%, Valid: 54.93% Test: 53.74%
Epoch: 500, Loss: 1.7074, Train: 56.07%, Valid: 57.08% Test: 56.16%
Epoch: 600, Loss: 1.6141, Train: 57.65%, Valid: 58.39% Test: 57.54%
Epoch: 700, Loss: 1.5425, Train: 58.79%, Valid: 59.40% Test: 58.48%
Epoch: 800, Loss: 1.4862, Train: 59.67%, Valid: 60.18% Test: 59.26%
Epoch: 900, Loss: 1.4409, Train: 60.38%, Valid: 60.85% Test: 59.90%
Epoch: 1000, Loss: 1.4037, Train: 60.91%, Valid: 61.46% Test: 60.37%
Epoch: 1100, Loss: 1.3728, Train: 61.38%, Valid: 61.99% Test: 60.79%
Epoch: 1200, Loss: 1.3466, Train: 61.82%, Valid: 62.38% Test: 61.17%
Epoch: 1300, Loss: 1.3242, Train: 62.20%, Valid: 62.68% Test: 61.39%
Epoch: 1400, Loss: 1.3048, Train: 62.58%, Valid: 62.99% Test: 61.65%
Epoch: 1500, Loss: 1.2878, Train: 62.89%, V